In [1]:
import os
import pandas as pd

In [2]:
pron = ['ეს', 'ეგ', 'ის', 'ასეთი', 'ეგეთი', 
            'ისეთი', 'ამნაირი', 'მაგნაირი', 'იმნაირი',
            'ამისთანა', 'მაგისთანა', 'იმისთანა',
            'ამდენი', 'მაგდენი', 'იმდენი', 'თვით', 'თითოეული', 'ყოველი', 
            'ვიღაც', 'რაღაც', 'რომელიღაც', 'როგორიღაც', 'ვინმე', 'რამე', 'რომელიმე', 'როგორიმე', 
            'ზოგი', 'ზოგიერთი', 'არავინ', 'ვერავინ', 'ნურავინ', 'არაფერი', 'ვერაფერი', 'ნურაფერი', 
            'ვერა', 'ნურავინ', 'სხვები', 'სხვებიც', 'ყველამ', 'ყველას', 'სხვასაც', 'ერთის', 'ზოგს']


pron_adj = ['ყველა', 'არცერთი', 'ზოგი', 'ზოგმა', 'სხვა']
mixed_pron = ['მეორეს', 'ერთმა', 'მეორემ' ,'ორნი']


In [3]:
TRAIN_PATH = os.path.join('data', 'train')
TEST_PATH = os.path.join('data', 'test')

In [4]:
df = pd.read_orc(os.path.join(TRAIN_PATH, 'train_1.orc'), dtype_backend='pyarrow')

In [5]:
sent = '<CLS> მე და დისევს გახლდით იმ დროს პელევსის სახლში, კარგად გვესმოდა, მამაშენი რას გარიგებდა, პელევსის პალატს, ლაღად მორთულს ვეწვიეთ მაშინ, რომ ბრძოლად გვეხმო აქაველთა ქვეყნის ლაშქარი. <SEP>'

In [6]:
sent = sent.split()

In [7]:
def get_pre_verb_adj(row):
    # print(row)
    words = row.mod_words
    pos_tags = row.pos_tags
  
    indicies = []
    for idx, (pos, word) in enumerate(zip(pos_tags, words)):
        if pos == 'V' and idx != 0 and pos_tags[idx - 1] == 'A':
            if word.endswith('-მეთქი'):
                word = word[:-6]
            if word.endswith('-თქო'):
                word = word[:-4]
            if word not in {'ვარ', 'ხარ', 'არის', 'ვართ', 'ხართ', 'არიან', 
                            'ვიყავით', 'იყავით', 'იყვნენ', 'ვიყავი', 'იყავი', 'იყო',
                            'ვიქნებოდი', 'იქნებოდი', 'იქნებოდა', 'ვიქნებოდით', 'იქნებოდით', 'იქნებოდნენ',
                            'ვიქნებოდე', 'იქნებოდე', 'იქნებოდა', 'ვიქნებოდეთ', 'იქნებოდეთ',
                            'ვიქნები', 'იქნები', 'იქნება', 'ვიქნებით', 'იქნებით',
                            'იქნება', 'ითვლება', 'იქცაო', 'იქცა', 'გადავიქეცი', 'გადაიქეცი', 'დავრჩი', 
                            'დარჩა', 'დარჩი', 'დავრჩით', 'დარჩით', 
                            'დარჩნენ'} and words[idx-1][-1] != 'ი' and words[idx-1][-1] != 'ს' and words[idx-1][:-2] != 'ია':
                indicies.append(idx - 1)
    

    return ['Adv' if idx in set(indicies) else pos for idx, pos in enumerate(pos_tags)]
    

In [8]:
def get_pronouns(row):
    words = row.mod_words
    pos_tags = row.pos_tags
    indicies = []
    for idx, (pos, word) in enumerate(zip(pos_tags, words)):
        if word.endswith('-მეთქი'):
            word = word[:-6]
        if word.endswith('-თქო'):
            word = word[:-4]
        if word in pron:
            indicies.append(idx)
        if ((len(pos_tags) != idx + 1) and pos_tags[idx+1] not in ('N', 'A')) and word in pron_adj:
            indicies.append(idx)
        if (len(pos_tags) != idx + 1) and words[idx+1] != 'მხრივ' and (pos_tags[idx+1] not in ('N', 'A') or words[idx+1][-1] == 'ი') and word in mixed_pron:
            indicies.append(idx)
    # print(indicies)
    return ['Pron' if idx in set(indicies) else pos for idx, pos in enumerate(pos_tags)]
        

In [84]:
df['pos_tags'] = df.apply(get_pre_verb_adj, axis=1)
df['pos_tags'] = df.apply(get_pronouns, axis=1)

df['pos_tags'] 

0                          [V, Pron, Pron, A, Cj, A, Punct]
1         [N, Adv, V, Punct, Cj, Pron, V, A, Cj, Adv, V,...
2                         [Punct, N, Cj, N, Pron, V, Punct]
3         [A, N, Punct, A, N, A, Cj, N, Punct, A, Punct,...
4                                   [Adv, V, Num, V, Punct]
                                ...                        
669512                               [N, V, A, A, N, Punct]
669513               [A, N, Cj, A, N, Cj, N, Adv, V, Punct]
669514         [Adv, Punct, Pron, N, V, N, N, N, Pp, Punct]
669515    [N, Cj, Pron, Pron, N, N, Punct, Pron, A, N, P...
669516                               [N, N, N, N, V, Punct]
Name: pos_tags, Length: 669517, dtype: object

In [9]:
for i in range(1, 10):
    df = pd.read_orc(os.path.join(TRAIN_PATH, f'train_{i}.orc'), dtype_backend='pyarrow')
    df['pos_tags'] = df.apply(get_pre_verb_adj, axis=1)
    df['pos_tags'] = df.apply(get_pronouns, axis=1)
    df.to_parquet(os.path.join(TRAIN_PATH, f'train_upd_{i}.parquet'))

In [10]:
for i in range(1, 10):
    df = pd.read_orc(os.path.join(TEST_PATH, f'test_{i}.orc'), dtype_backend='pyarrow')
    df['pos_tags'] = df.apply(get_pre_verb_adj, axis=1)
    df['pos_tags'] = df.apply(get_pronouns, axis=1)
    df.to_parquet(os.path.join(TEST_PATH, f'test_upd_{i}.parquet'))

In [97]:
print('Done')

Done


In [38]:
def get_adv_v(row):
    words = row.mod_words
    indicies = row.inc_adj
    pairs = []
    for idx in indicies:
        pairs.append((words[idx], words[idx + 1]))
    return pairs
    

In [39]:
df['pairs'] = df.apply(get_adv_v, axis=1)

In [42]:
# df[df['pairs'].apply(lambda l: len(l) != 0)]['pairs'].tolist()